# Chain of tasks:

In [1]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
import os, json
from maestro.api import APIClient, Provider
from maestro.api import Dataset, Image, Task, Group
from maestro.schemas import TaskInputs

In [3]:
token="fcad6b3734b74e7e901c58903dc416f8c5029cf224714f518f3f04ac379d7b7a"
host="http://caloba51.lps.ufrj.br:7000"
basepath = os.getcwd()

## Create remote session:

In [4]:
provider = Provider(host=host, token=token)

## create the dataset:

In [5]:
dataset_evt = Dataset(name="user.joao.pinto.mc25_13TeV.JF17.EVT.r1000")

In [6]:
dataset_evt.create()

2025-03-28 08:45:25.778 | DEBUG    | maestro.api.rest.dataset:create:104 - Received dataset_id: 8164a3e141d2


'8164a3e141d2'

In [8]:
path='/mnt/cern_data/joao.pinto/lorenzetti/mc25_13TeV/user.joao.pinto.mc25_13TeV.JF17.EVT.r1000'
dataset_evt.upload(path,as_link=True)

uploading file...: 100%|██████████████████████████████████████████| 200/200 [00:20<00:00,  9.86it/s]


True

In [9]:
dataset_evt.print()

Dataset name : user.joao.pinto.mc25_13TeV.JF17.EVT.r1000
dataset id   : 8164a3e141d2
+-------------------+----------------------------------+
| filename          | md5                              |
|-------------------+----------------------------------|
| JF17.EVT.12.root  | 1a074360a5b1a0e653fb24ead7c0b394 |
| JF17.EVT.8.root   | a738e06582a86729aa7e9341919c4efa |
| JF17.EVT.21.root  | 8821839c78859aad7a1c53104239c616 |
| JF17.EVT.7.root   | b58f9c61f57f6e88400625e660f8bcac |
| JF17.EVT.9.root   | e2a581e5753b60113a2e60777c67af44 |
| JF17.EVT.0.root   | ba198b3c193370bfe696e3470788e04c |
| JF17.EVT.30.root  | 44e047b166f5ee4dfbc22aa593860fb3 |
| JF17.EVT.6.root   | a80d28c21f67b2691a5f8e312a695ce5 |
| JF17.EVT.23.root  | b9cf683cc90b6a87d1ed4f53b05ddc80 |
| JF17.EVT.10.root  | 098a5f15cda05729d9ea3bd47af5febf |
| JF17.EVT.38.root  | f097d5fccf8d2de114da6832b19887e2 |
| JF17.EVT.4.root   | 2bdb0b8fdf75994eadea77bed48c0607 |
| JF17.EVT.24.root  | 2de4b2d88e6ae07f75bc7db2ab857932 |
| J

## Create the image:

In [10]:
image = Image("joao.pinto/lorenzetti_latest", "/mnt/cern_data/joao.pinto/images/lorenzetti_latest.sif", as_link=True)

## Create the task:

In [16]:
def production(name, partition, dataset):
    GB=1024
    build_dir = '/home/joao.pinto/git_repos/lorenzetti/build'
    preexec   = f"source {build_dir}/lzt_setup"

    task_stage_1          = TaskInputs( name=name+"_stage_1", image='joao.pinto/lorenzetti_latest' )
    task_stage_1.input    = dataset_evt.name
    task_stage_1.command  = f"{preexec} && simu_trf.py -i %IN -o %OUT"
    task_stage_1.outputs  = {"OUT":'result.HIT.root'}
    task_stage_1.envs     = {}
    task_stage_1.memory_mb=20*GB
    task_stage_1.cpu_cores=10
    task_stage_1.partition=partition
    
    dataset_hit_name      = task_stage_1.name+"."+task_stage_1.outputs["OUT"]
    task_stage_2          = TaskInputs( name=name+"_stage_2", image='joao.pinto/lorenzetti_latest' )
    task_stage_2.input    = dataset_hit_name
    task_stage_2.command  = f"{preexec} && digit_trf.py -i %IN -o %OUT"
    task_stage_2.outputs  = {"OUT":'result.ESD.root'}
    task_stage_2.envs     = {}
    task_stage_2.memory_mb=10*GB
    task_stage_2.cpu_cores=4
    task_stage_2.partition=partition

    
    dataset_esd_name      = task_stage_2.name+"."+task_stage_2.outputs["OUT"]
    task_stage_3          = TaskInputs( name=name+"_stage_3", image='joao.pinto/lorenzetti_latest' )
    task_stage_3.input    = dataset_esd_name
    task_stage_3.command  = f"{preexec} && reco_trf.py -i %IN -o %OUT"
    task_stage_3.outputs  = {"OUT":'result.AOD.root'}
    task_stage_3.envs     = {}  
    task_stage_3.memory_mb=10*GB
    task_stage_3.cpu_cores=4
    task_stage_3.partition=partition

    
    dataset_aod_name      = task_stage_3.name+"."+task_stage_3.outputs["OUT"]
    task_stage_4          = TaskInputs( name=name+"_stage_4", image='joao.pinto/lorenzetti_latest' )
    task_stage_4.input    = dataset_aod_name
    task_stage_4.command  = f"{preexec} && ntuple_trf.py -i %IN -o %OUT"
    task_stage_4.outputs  = {"OUT":'result.NTUPLE.root'}
    task_stage_4.envs     = {}  
    task_stage_4.memory_mb=10*GB
    task_stage_4.cpu_cores=4
    task_stage_4.partition=partition
    
    print("stage 1")
    print(f"task    : {task_stage_1.name}")
    print(f"image   : {task_stage_1.image}")
    print(f"input   : {task_stage_1.input}")
    print(f"command : {task_stage_1.command}")

    print("stage 2")
    print(f"task    : {task_stage_2.name}")
    print(f"image   : {task_stage_2.image}")
    print(f"input   : {task_stage_2.input}")
    print(f"command : {task_stage_2.command}")

    print("stage 3")
    print(f"task    : {task_stage_3.name}")
    print(f"image   : {task_stage_3.image}")
    print(f"input   : {task_stage_3.input}")
    print(f"command : {task_stage_3.command}")

    print("stage 4")
    print(f"task    : {task_stage_4.name}")
    print(f"image   : {task_stage_4.image}")
    print(f"input   : {task_stage_4.input}")
    print(f"command : {task_stage_4.command}")
    
    
    return [task_stage_1,task_stage_2,task_stage_3,task_stage_4]


task_name = 'user.joao.pinto.mc25_13TeV.JF17.r1000'
partition = 'cpu-large'
tasks     = production(task_name, partition, dataset_evt) 

stage 1
task    : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_1
image   : joao.pinto/lorenzetti_latest
input   : user.joao.pinto.mc25_13TeV.JF17.EVT.r1000
command : source /home/joao.pinto/git_repos/lorenzetti/build/lzt_setup && simu_trf.py -i %IN -o %OUT
stage 2
task    : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_2
image   : joao.pinto/lorenzetti_latest
input   : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_1.result.HIT.root
command : source /home/joao.pinto/git_repos/lorenzetti/build/lzt_setup && digit_trf.py -i %IN -o %OUT
stage 3
task    : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_3
image   : joao.pinto/lorenzetti_latest
input   : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_2.result.ESD.root
command : source /home/joao.pinto/git_repos/lorenzetti/build/lzt_setup && reco_trf.py -i %IN -o %OUT
stage 4
task    : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_4
image   : joao.pinto/lorenzetti_latest
input   : user.joao.pinto.mc25_13TeV.JF17.r1000_stage_3.result.AOD.root
command : source

In [18]:
my_task=provider.create(tasks)

In [19]:
my_task.print()

+------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------+
| taskname                     |   registered |   assigned |   pending |   running |   failed |   kill |   killed |   broken |   completed | Status    |
|------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------|
| user.joao.pinto.simple_task3 |            0 |          0 |         0 |         0 |        0 |      0 |        0 |        0 |          10 | completed |
+------------------------------+--------------+------------+-----------+-----------+----------+--------+----------+----------+-------------+-----------+


In [20]:
provider.print_datasets()

+--------------+------------------------------------------+---------+--------+
| id           | name                                     |   files | type   |
|--------------+------------------------------------------+---------+--------|
| 6468a74db763 | user.joao.pinto.dataset                  |      10 | files  |
| 55f87ba95f53 | joao.pinto/python3.10                    |       1 | image  |
| 4bf61d1215df | user.joao.pinto.simple_task3.result.json |      10 | files  |
+--------------+------------------------------------------+---------+--------+


In [21]:
provider.print_tasks()

+--------------+------------------------------+-----------+
| id           | name                         | type      |
|--------------+------------------------------+-----------|
| 0f789a0f2c04 | user.joao.pinto.simple_task3 | completed |
+--------------+------------------------------+-----------+
